In [1]:
import os
dir_notebook = os.getenv( "HOME" )
file_include = dir_notebook + "/_include.ipynb"
%run $file_include

file manipulation

In [2]:
dir_diamond = dir_notebook + "/diamond-0.9.3/"
dir_CAU = "/media/sf_j1/MAP/analyzed_data_new/BPGA/MAP_40/identity_0.9/Sequences/"
lst_genome = os.listdir( dir_CAU )
lst_genome.sort()
print lst_genome

['REPSEQ_ACCESSORY.txt', 'REPSEQ_CORE.txt', 'REPSEQ_UNIQUE.txt', 'accessory_seq.txt', 'core_seq.txt', 'exclusively_absent_seq.txt', 'unique_seq.txt']


In [3]:
dir_genome = "/media/sf_j1/MAP/analyzed_data_new/COG/MAP_40/identity_0.9/genome/"
for x in lst_genome:
    if "REPSEQ" not in x:
        continue
    f_name = dir_genome + x.replace('.txt', '.faa')
    f = open(f_name, 'w')
    for y in SeqIO.parse(dir_CAU + x, 'fasta'):
        ID = y.id
        if "|" in ID:
            ID = ID.split('|')[-1]
        if "/" in ID:
            ID = ID.split('/')[-1]
        ID = ID.replace("#", "")
        f.write('>' + ID + '\n')
        f.write( str(y.seq) + '\n')
    f.close()


COG analysis

In [4]:
dir_cog = dir_notebook + "/diamond-0.9.3/"
path_diamond = dir_cog + "diamond"
path_db_cog = dir_cog + "prot2003-2014.dmnd" 

lst_files = os.listdir( dir_genome )
print lst_files

['REPSEQ_ACCESSORY.faa', 'REPSEQ_CORE.faa', 'REPSEQ_UNIQUE.faa']


In [5]:
from IPython.display import clear_output
identity = 80

for file0 in lst_files :
    path_faa = dir_genome + "%s" % file0
    path_out = dir_genome.replace("genome", "cog") + "cog_%s" % file0
    path_out = path_out.replace( ".faa", ".out" )

    COG_command = "%s blastp --db %s --query %s --out %s --id %s" % (
                    path_diamond, path_db_cog, path_faa, path_out, identity)
    
    print COG_command
    !$COG_command
    #clear_output()
    break

/home/j1lim/diamond-0.9.3/diamond blastp --db /home/j1lim/diamond-0.9.3/prot2003-2014.dmnd --query /media/sf_j1/MAP/analyzed_data_new/COG/MAP_40/identity_0.9/genome/REPSEQ_ACCESSORY.faa --out /media/sf_j1/MAP/analyzed_data_new/COG/MAP_40/identity_0.9/cog/cog_REPSEQ_ACCESSORY.out --id 80
diamond v0.9.3.104 | by Benjamin Buchfink <buchfink@gmail.com>
Licensed under the GNU AGPL <https://www.gnu.org/licenses/agpl.txt>
Check http://github.com/bbuchfink/diamond for updates.

#CPU threads: 4
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
#Target sequences to report alignments for: 25
Temporary directory: /media/sf_j1/MAP/analyzed_data_new/COG/MAP_40/identity_0.9/cog
Opening the database...  [3.2e-05s]
Opening the input file...  [0.002788s]
Opening the output file...  [0.002238s]
Loading query sequences...  [0.014978s]
Masking queries...  [0.064384s]
Building query seed set...  [0.005417s]
Algorithm: Query-indexed
Building query histograms...  [0.000607s]
Allocating

In [6]:
path_cogname = "/home/j1lim/prot2003-2014/cognames2003-2014.tab"
df_cogname = pd.read_csv( path_cogname, sep = "\t" )
df_cogname.columns = [ "cog", "function", "name" ]

dic_cogfunction = { row.cog: row.function for idx, row in df_cogname.iterrows() }
dic_cogname = { row.cog: row[2] for idx, row in df_cogname.iterrows() }

print len( dic_cogfunction )
print len( dic_cogname )

path_cog = "/home/j1lim/prot2003-2014/cog2003-2014.csv"
df_cog = pd.read_csv( path_cog, header = -1 )
df_cog.columns = [ "domain_id", "genome_name", "protein_id", "protein_length", "domain_start", "domain_end", "cog_id", 
                  "membership_class", "empty" ]

print len( df_cog )

dic_cog = {}
dic_protein_cog = {}
for idx, row in df_cog.iterrows() :
    protein_id = row.protein_id
    if protein_id not in dic_cog :
        dic_cog[ protein_id ] = ""
        dic_protein_cog[ protein_id ] = ""
    if row.cog_id not in dic_cogfunction :
        continue
    dic_cog[ protein_id ] = dic_cog[ protein_id ] + dic_cogfunction[ row.cog_id ]
    dic_protein_cog[ protein_id ] = dic_protein_cog[ protein_id ] + "," + row.cog_id
    
for k, v in dic_cog.items() :
    cog = "".join( set( v ) )
    dic_cog[ k ] = cog


print len( dic_cog ), len( dic_protein_cog )

4631
4631
1962317
1785722 1785722


In [7]:
#lst_files = [ "FBL1" ]

for file0 in lst_files :
    path_faa = dir_genome + "%s" % file0
    
    dic_anno = {}
    for seq in SeqIO.parse(path_faa, "fasta"):
        dic_anno[seq.id] = str(seq.seq)
    lst_genes = dic_anno.keys()
    
    path_out = dir_genome.replace("genome", "cog") + "cog_%s" % file0.replace('faa', 'out')
    
    df_blast = pd.read_csv( path_out, sep = "\t", header = -1 )
    df_blast.columns = [ "gene_id", "target", "pidentity", "length", "mismatch", "gapopen", "qstart", 
                        "qend", "sstart", "send", "evalue", "bitscore" ]

    path_gene_cog = dir_genome.replace("genome", "gene_cog") +"%s" % file0
    path_gene_cog = path_gene_cog.replace( ".faa", ".out" )
    path_gene_table = dir_genome.replace("genome", "tables") + "table_%s" % file0.replace( 'faa', 'out' )
    o_cog = open( path_gene_cog, "w" )
    o_table = open( path_gene_table, 'w' )
    
    #print df_blast.head( 5 )
    dic_blast = {}
    for idx, row in df_blast.iterrows() :
        if row.gene_id not in dic_blast :
            dic_blast[ row.gene_id ] = row
    print file0, len( dic_blast )
    
    
    dic_cog0 = {}
    for gene_id in lst_genes :
        if gene_id not in dic_blast :
            o_cog.write( "%s\tS\r\n" % gene_id )
            
            o_table.write( ">%s\t\r\n" % gene_id )
            continue
            
        row_blast = dic_blast[ gene_id ]
    
        protein_id = int( row_blast.target.split( "|" )[ 1 ] )
        if protein_id not in dic_cog :
            o_cog.write( "%s\tS\r\n" % gene_id )
            
            o_table.write( ">%s\t\r\n" % gene_id )
            continue
            
        cog_group = dic_cog[ protein_id ]

        dic_cog0[ protein_id ] = cog_group
        o_cog.write( "%s\t%s\r\n" % (gene_id, cog_group) )
        
        lst_cog_id = (dic_protein_cog[ protein_id ]).split(",")
        lst_cog_id = lst_cog_id[1:]
        
        o_table.write( ">%s" % gene_id )
        for l in lst_cog_id:
            o_table.write( "\t\t%s\t%s\t%s\t%s\t\n" % (str(protein_id), l, str(dic_cogfunction[l]), str(dic_cogname[l])) )
    print len( dic_cog0 )
    
    lst_cog = []
    for k, v in dic_cog0.items() :
        lst_cog.extend( list( v ) )
        
    o_cog.close()
    o_table.close()
    
    import collections
    c = collections.Counter( lst_cog )
    print c

REPSEQ_ACCESSORY.faa 133
118
Counter({'R': 19, 'I': 14, 'H': 12, 'C': 11, 'J': 11, 'L': 9, 'Q': 9, 'E': 8, 'M': 8, 'F': 6, 'O': 6, 'G': 5, 'X': 5, 'T': 4, 'V': 4, 'K': 3, 'S': 3, 'P': 2, 'U': 2, 'D': 1, 'N': 1, 'W': 1})
REPSEQ_CORE.faa 3240
3074
Counter({'R': 486, 'I': 426, 'K': 285, 'Q': 281, 'H': 247, 'C': 237, 'E': 226, 'P': 216, 'S': 199, 'M': 184, 'G': 176, 'J': 175, 'T': 150, 'O': 113, 'L': 110, 'V': 94, 'F': 64, 'D': 48, 'U': 20, 'X': 14, 'N': 13, 'W': 3, 'A': 2, 'B': 1})
REPSEQ_UNIQUE.faa 299
242
Counter({'R': 34, 'I': 32, 'Q': 28, 'X': 25, 'S': 22, 'H': 19, 'M': 16, 'T': 14, 'V': 14, 'C': 13, 'E': 13, 'L': 13, 'G': 12, 'O': 10, 'P': 10, 'K': 9, 'J': 8, 'U': 5, 'F': 4, 'D': 3, 'N': 2, 'W': 1})


In [8]:
dir_genecog = "/media/sf_j1/MAP/analyzed_data_new/COG/MAP_40/identity_0.9/gene_cog/"
file_genecog = os.listdir(dir_genecog)
print file_genecog

['REPSEQ_ACCESSORY.out', 'REPSEQ_CORE.out', 'REPSEQ_UNIQUE.out']


In [9]:
for x in file_genecog:
    if "REPSEQ_" not in x:
        continue
    print x, "/",
    seq_ = ""
    f_ = file(dir_genecog + x)
    r_ = f_.read()
    f_.close()
    spl = r_.split("\r\n")
    gene_num = 0
    for y in spl:
        if y == "":
            continue
        lst_ = y.split("\t")
        seq_ += lst_[1]
        gene_num += 1 
        #break
    print "gene_num: ", gene_num, "/", "cog_num", len(seq_)
    print collections.Counter( seq_ )
    print seq_
    #break

REPSEQ_ACCESSORY.out / gene_num:  1268 / cog_num 1297
Counter({'S': 1138, 'R': 19, 'I': 17, 'H': 13, 'J': 13, 'C': 11, 'E': 11, 'M': 11, 'L': 11, 'Q': 11, 'F': 7, 'O': 6, 'G': 5, 'X': 5, 'T': 4, 'V': 4, 'K': 3, 'U': 3, 'P': 2, 'D': 1, 'N': 1, 'W': 1})
SSSSSCSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSGSSSSISSIQSSSSSSSSSSSSSOSSSSSSSSSSSSSSSSSSSSSTSSSSSSSSSSSLSSSSSSSSSSRSSSSSSSISSSSSSSSSSSSSIQSSSSSSSSSSSSSSSHSSSSSSSSSSSSSSSSSSSSHSSSSSSSSSFSSSSSSSSSSSSSSSSSSSSSSLSSSSSSSSSSMSSSSSSSUSSSSSSSSSLSSSSSSSSIQRSSSSSSSSSSSSSSSSLSSSRSSSSSXSSJSSSSSSSSSSSSISSSSSSSSSSSESSSPSSSISSSSSISSSSSSSSSSSSSSFSSSSSSSSSSSSSSSSJSSTSSSSSSSSSSSSRSSSIQCRSSHRJHSSSSSSSSSSSSSSSSSSSDSSIRLJSSSSSSSSSSSSSLSSSSSSSSMVSSMSSSSSSSSSSSSXSSSSSSLSSSSSSSSSSKKSSSFUSSSOSSSSHSJCSSSSSSSISSSSSSMSCSSSSSHRSSSSSSSSSSSSSSEISSSSFSSSSSSSSSSSSSSSSSSSSSSESSSJSHRSSSSHRSSSSSSSSJSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSQESHSSRSSSOSSSSSSSSSSSMSESSSESSSSSSSSSSSSISSSSSSSSSSHESSSSIRSSHSSSSMSSSSSSSSSSSSSJPSSSSSSSSSSSSSSSSHCSSSSSSSSSSSSSSSSSSSMSSSSTSSSSSSSSGSSSSSSSSSSSS

In [14]:
dir_genecog = "/media/sf_j1/MAP/analyzed_data_new/COG/MAP_40/identity_0.9/_identity_0.8/gene_cog/"
file_genecog = os.listdir(dir_genecog)
print file_genecog

['REPSEQ_ACCESSORY.out', 'REPSEQ_CORE.out', 'REPSEQ_UNIQUE.out']


In [15]:
for x in file_genecog:
    if "REPSEQ_" not in x:
        continue
    print x, "/",
    seq_ = ""
    f_ = file(dir_genecog + x)
    r_ = f_.read()
    f_.close()
    spl = r_.split("\r\n")
    gene_num = 0
    for y in spl:
        if y == "":
            continue
        lst_ = y.split("\t")
        seq_ += lst_[1]
        gene_num += 1 
        #break
    print "gene_num: ", gene_num, "/", "cog_num", len(seq_)
    print collections.Counter( seq_ )
    print seq_
    #break

REPSEQ_ACCESSORY.out /

IndexError: list index out of range